**Note:** This notebook is written in the Julia language, so the cells can't be
  executed in Google Colab. If you want to verify that the notebook works, I
  recommend [JuliaBox](https://juliabox.com/) or testing locally. The syntax is
  very similar to Python and MATLAB. Note in particular the dot syntax used to
  perform elementwise operations (`f.(x)` applies `f` to all elements of `x`)
  and that indices start at 1.

# **Lab 2: Direct methods**
**Anders Ågren Thuné**

# **Abstract**

Short summary of the lab report. State the objectives, methods used, main
results and conlusions.

# *About the code**

A short statement on who is the author of the file, and if the code is
distributed under a certain license.

In [2]:
"""
DD2363 Methods in Scientific Computing,
KTH Royal Institute of Technology, Stockholm, Sweden.
"""

# Copyright (C) 2019
# Anders Ågren Thuné (athune@kth.se)
# Johan Hoffman (jhoffman@kth.se)

# Code written by Anders Ågren Thuné based on the template by Johan Hoffman.

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

"DD2363 Methods in Scientific Computing,\nKTH Royal Institute of Technology, Stockholm, Sweden.\n"

# **Set up environment**

In [3]:
using LinearAlgebra
import Base: getproperty

# **Introduction**

Systems of linear equations appear frequently in a wide variety of problems. A
system of linear equations can be expressed in linear algebra terms as $Ax = b$,
which, if solvable, has the solution $x = A^{-1}b$. As such, being able to
compute the inverse of a given matrix is a problem of great importance. This is
difficult in general, but certain classes of matrices have easily computable
inverses; an orthogonal matrix $Q$ has the inverse $Q^T$ and the inverse of a
triangular matrix can be computed through back (or forward) substitution.
Therefore, a common approach to computing the inverse of a given matrix is to
factor it into triangular and orthogonal matrices, which can then be easily
inverted. One such factorization is the QR factorization in which any matrix $A$
is factored into an orthogonal matrix $Q$ and an upper triangular matrix $R$.
The inverse is then given by $A^{-1}=(QR)^{-1}=R^{-1}Q^T$.

Even when the system $Ax=b$ is unsolvable (there is no $A^{-1}), the best
possible solution can be obtained by projecting $b$ orthogonally onto
$range(A)$. This gives the approximated solution $\hat{x} = (A^TA)^{-1}A^Tb$,
where $(A^TA)^{-1}$ is called the *pseudo-inverse* of $A$.

The QR factorization can also be used in a wider range of problems. One such
problem is eigenvalue computation, where an iterative $QR$-algorithm can be used
to obtain a Schur factorization of a symmetric matrix. That is, a factorization
$A = QTQ^T$, where $T$ is a triangular matrix with the eigenvalues of $A$ on
the diagonal.

This report presents how these concepts, described in chapters 5 and 6 of the
lecture notes, were used to implement the following:
- A function for QR factorization
- A direct solver of of $Ax=b$
- A solver of the least squares problem $Ax=b$
- A function performing the QR eigenvalue algorithm

# **Methods**

## **QR factorization**

The QR factorization for a matrix can be calculated in a number of different
ways. The method implemented here is the *Householder QR factorization*, based
on Householder reflections. A matrix $P$ of the form $P = I-\beta vv^T, \quad
\beta = \frac{2}{v^Tv}$ is a *Householder reflection*. This is an orthogonal
reflector reflecting a given vector $x$ in the hyperplane
$\text{span}\{v\}^\perp. Selecting $v$ = $\pm \|x\|$ gives $Px = \pm\|x\|e_1$,
which can be utilized to construct $Q_n\dots Q_2Q_1A = R$, where each matrix
$Q_k$ is constructed to zero the subdiagonal elements of a column of $A$. This
is achieved by letting $Q_k = \begin{matrix} I & 0 \\ 0 P \end{matrix}$, where P
is a Householder reflection. When dealing with a Householder QR factorization,
it is advantageous not to explicitly form the matrices $Q$ or $Q_k$, but instead
utilize that $Q_kA = A - (\beta v)(v^TA)$ and $Q_kA = A - (A v)(\beta
v)^T$.

[Golub & Van Loan 2013: Chapter 5.1-5.2, Hoffman 2019: Chapter 5.2]

The following implementation is based on Algorithm 5.2.1 of Golub & Van Loan
(2013), and results in a matrix with the upper triangular part of $R$ as its
upper triangular part, and the vectors $v_j$ required to construct $Q$ below the
diagonal. To be precise, all elements except the first of these vectors are
stored, with the implicit assumption that they are normalized such that the
first element is always 1. In addition to the function itself, a wrapper
struct (analogous to a Python class) is implemented to facilitate easy handling
of this format.

In [242]:
"""
A struct for representing the QR factorization of a matrix
in a compact way. The constructor computes the factorization
using Algorithm 5.2.1 of Golub & Van Loan (2013),
modifying the argument in place.
"""
mutable struct QRfact{T <: AbstractFloat}
    QR :: Matrix{T}
    betas :: Vector{T}

    function QRfact(A :: Matrix{T}) where T <: AbstractFloat
        (m, n) = size(A)
        v = zeros(T, m)
        betas = zeros(T, n)

        for j = 1 : n
            v[j:m] .= A[j:m,j]
            v[j] += sign(v[j])*norm(v[j:m])
            betas[j] = 2v[j]^2/norm(v[j:m])^2
            v[j:m] ./= v[j]
            for k = j : n
                A[j:m,k] .-= betas[j].*v[j:m].*dot(v[j:m],A[j:m,k])
            end
            if j<m
               A[j+1:m,j] .= v[j+1:m]
            end
        end
        new{T}(A, betas)
    end
end

"""
Overloads getproperty to allow easy extraction of Q and R factors.
Enables dot syntax, as in the following example:
> QR = QRfactorization(A)
> Q = QR.q
> R = QR.r
"""
function getproperty(qr :: QRfact{T}, f :: Symbol) where T
    QR = getfield(qr, :QR)
    m = size(QR)[1]
    if f == :q
        Q = Matrix{T}(I,m,m)
        mulQ!(qr,Q)
        return Q
    elseif f == :qt
        Qt = Matrix{T}(I,m,m)
        mulQ!(qr,Qt,transpose=true)
        return Qt
    elseif f == :r
        return UpperTriangular(copy(QR))
    elseif f == :r_ref
        return UpperTriangular(QR)
    else
        error("type $(typeof(qr)) has no field $f")
    end
end

"""
Multiply the matrix C with the Q factor of a QR factorization.
Keywords specify whether to multiply with Q or Q^T, and whether
to multiply from the left or from the right. Modifies C in place.
"""
function mulQ!(qr :: QRfact{T}, C :: VecOrMat{T};
               transpose = false, fromright = false) where T
    QR = getfield(qr, :QR)
    betas = getfield(qr, :betas)
    (m,n) = size(QR)
    Csize = typeof(C) <: Vector ? (length(C), 1) : size(C)

    match_rowcol = Int(!xor(transpose, fromright))+1
    if Csize[match_rowcol] != m
        throw(DomainError((m,Csize[match_rowcol]),
            "The matrices' dimensions don't match!"))
    end
    iterRange = [(1:n),(n:-1:1)][match_rowcol]

    v = zeros(T, m)
    temp = zeros(T,m)
    for j = iterRange
        v[j] = one(T)
        v[j+1:m] .= QR[j+1:m,j]
        if fromright
            mul!(temp, C[:,j:m], v[j:m])
            for k = j : m
                C[:,k] .-= temp .*betas[j].*v[k]
            end
        else
            for k = 1 : Csize[2]
                C[j:m,k] .-= betas[j].*v[j:m].*dot(v[j:m],C[j:m,k])
            end
        end
    end
end

"""
Get the QR factorization of A without modifying the original.
"""
QRfactorization(A :: Matrix{<:AbstractFloat}) = QRfact(copy(A))

QRfactorization

## **Direct solver**

The solution to the system $Ax=b$ can be obtained as $x = A^{-1}b = R^{-1}Q^Tb$,
once $A$ has been factored into $Q$ and $R$. Setting $y$ = $Q^Tb$, the equation
to solve is $x = R^{-1}y$. Algorithm 2 of chapter 5 in the notes describes how
such a system can be solved, using *back substitution*. In the implementation
below, the summation of the original algorithm is replaced by a dot product.

In [5]:
"""
Compute x = U^-1 b through back substitution for any upper triangular matrix U.
"""
function backsubstitute(U::UpperTriangular, b::Vector)
    m = length(b)
    if m != size(U)[1]
        throw(DomainError((m,size(U)[1]), "The dimensions of U and b don't match!"))
    end
    x = zeros(m)
    x[m] = b[m]/U[m,m]
    for j = m-1:-1:1
        x[j] = (b[j] - dot(U[j,j+1:m],x[j+1:m]))/U[j,j]
    end
    x
end

backsubstitute

In [6]:
"""
Compute x = A^-1 b for any invertible matrix A.
"""
function directsolver(A::Matrix, b::Vector)
    qr = QRfactorization(A)
    y = copy(b)
    mulQ!(qr,y, transpose=true)
    backsubstitute(qr.r_ref,y)
end

directsolver

## **Least squares solver**

This is very similar to the above. In order to solve the least squares problem,
we solve $x = (A^TA)^{-1}(A^Tb)$, which is an equation solvable directly, as
$A^TA$ is a positive definite symmetric matrix for all $A$.

In [7]:
"""
Compute the best solution x to Ax = b for a non-invertible matrix A, given by
x = (A^T A)^-1 (A^T b)
"""
function leastsquares(A::Matrix, b::Vector)
    directsolver(A'A, A'b)
end

leastsquares

In [228]:
A = rand(6,3)

6×3 Array{Float64,2}:
 0.331104   0.37678    0.439453
 0.134874   0.657737   0.159695
 0.991759   0.915121   0.82592 
 0.81924    0.0765112  0.962414
 0.0956698  0.0057181  0.642411
 0.643241   0.882871   0.432216

In [229]:
b = rand(6)

6-element Array{Float64,1}:
 0.9758922386462052 
 0.09764567162881077
 0.4148709981344445 
 0.6367548349105641 
 0.5202630674254234 
 0.6521411952019074 

In [243]:
test = qr(A)

LinearAlgebra.QRCompactWY{Float64,Array{Float64,2}}
Q factor:
6×6 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.222953   -0.126732    0.25156    -0.580218   -0.695514   -0.224846 
 -0.0908191  -0.61353     0.274618    0.638328   -0.192411   -0.308923 
 -0.667814   -0.141745   -0.158279   -0.223314    0.512283   -0.443371 
 -0.551646    0.640008    0.0649479   0.439115   -0.295653    0.0403279
 -0.0644205   0.0783239   0.898135   -0.10977     0.353326    0.214897 
 -0.433135   -0.414348   -0.159332   -0.0338069  -0.0674942   0.780785 
R factor:
3×3 Array{Float64,2}:
 -1.48508  -1.17985   -1.42354 
  0.0      -0.897405   0.216441
  0.0       0.0        0.59429 

In [244]:
f = getfield(test.Q, :factors)

6×3 Array{Float64,2}:
 -1.48508    -1.17985    -1.42354 
  0.0742621  -0.897405    0.216441
  0.546067    0.0452218   0.59429 
  0.451077   -0.434615   -0.047971
  0.0526762  -0.0529884  -0.687294
  0.354171    0.230321    0.235136

In [245]:
test2 = QRfactorization(A);
getfield(test2, :QR)

6×3 Array{Float64,2}:
 -1.48508    -1.17985    -1.42354 
  0.0742621  -0.897405    0.216441
  0.546067    0.0452218   0.59429 
  0.451077   -0.434615   -0.047971
  0.0526762  -0.0529884  -0.687294
  0.354171    0.230321    0.235136

In [246]:
r = copy(getfield(test2,:QR))
for i in 1:3
r[i+1:end, i] .= 0
end
r

6×3 Array{Float64,2}:
 -1.48508  -1.17985   -1.42354 
  0.0      -0.897405   0.216441
  0.0       0.0        0.59429 
  0.0       0.0        0.0     
  0.0       0.0        0.0     
  0.0       0.0        0.0     

In [247]:
A

6×3 Array{Float64,2}:
 0.331104   0.37678    0.439453
 0.134874   0.657737   0.159695
 0.991759   0.915121   0.82592 
 0.81924    0.0765112  0.962414
 0.0956698  0.0057181  0.642411
 0.643241   0.882871   0.432216

In [248]:
test2.q*r

6×3 Array{Float64,2}:
 0.331104   0.37678    0.439453
 0.134874   0.657737   0.159695
 0.991759   0.915121   0.82592 
 0.81924    0.0765112  0.962414
 0.0956698  0.0057181  0.642411
 0.643241   0.882871   0.432216

In [249]:
Ass = copy(A)
mulQ!(test2, Ass, transpose=true)
Ass

6×3 Array{Float64,2}:
 -1.48508      -1.17985      -1.42354    
 -1.41108e-16  -0.897405      0.216441   
  1.91642e-16   4.07568e-17   0.59429    
 -4.25567e-16   1.08401e-16   0.0        
 -4.76084e-16  -2.36722e-17   0.0        
 -2.08486e-16  -4.26646e-17   2.77556e-17

In [250]:
A\b

3-element Array{Float64,1}:
 -0.45182365497849203
  0.31979395598773414
  1.02872362691589   

In [251]:
leastsquares(A,b)

3-element Array{Float64,1}:
 -0.45182365497849  
  0.3197939559877336
  1.028723626915888 

# **Results**

Present the results. If the result is an algorithm that you have described under
the *Methods* section, you can present the data from verification and
performance tests in this section. If the result is the output from a
computational experiment this is where you present a selection of that data.

# **Discussion**

Summarize your results and your conclusions. Were the results expected or
surprising. Do your results have implications outside the particular problem
investigated in this report?

# **References**

- Hoffman, J. 2019. *Introduction to Scientific Computing*
- Golub, Gene H. and Van Loan, Charles F. 2013. *Matrix Computations*. 4th ed. Baltimore: John Hopkins University Press.